In [9]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library("stringr")

In [10]:
# alpha = 1, beta for the L1 loss of the guide embeddings varies: "model_0", "model_000001", "model_00001", "model_0001", "model_001", "model_01", "model_1"
# alpha =1, beta = 0, theta varies for the multilabel cost varies: "model_00_multilab", "model_000_multilab", "model_0000_multilab"
# alpha =1, beta varies again but a model where guide embeddings are only involved in the decoder : "model2_0", "model2_000001", "model2_0001", "model2_001", "model2_01", "model2_1"
# beta = 0, alpha varies: "model_alpha001", "model_alpha005", "model_alpha05", "model_alpha1", "model_alpha3", "model_alpha5", "model_alpha20"

models= read.csv(paste0(projectDir, "./TextFiles/ComboKOModels.csv"))
head(models)

,Model_Name,Alpha,ComboIncluded,SubFolder
,<chr>,<dbl>,<int>,<chr>
1,model_alpha_0_1_v1,0.1,0,K_singles
2,model_alpha_0_1_v2,0.1,0,K_singles
3,model_alpha_0_1_v3,0.1,0,K_singles
4,model_alpha_0_1_v4,0.1,0,K_singles
5,model_alpha_0_1_v5,0.1,0,K_singles
6,model_alpha_0_1_v6,0.1,0,K_singles


In [11]:
combosObserved <- readRDS(paste0(projectDir, "./RDSFiles/ComboEffects_lm_residuals.rds"))
combosObserved <- combosObserved[combosObserved$term %ni% c("(Intercept)", "n_genes", "mt_frac", paste0("leiden", 1:9)),]
combosObserved[,c("std.error", "statistic", "p.value")] = NULL
combosObserved$term <- sapply(combosObserved$term, function(x){str_replace_all(x, "_", "")})
colnames(combosObserved) = c("Group", "ObservedLogFCEstimate", "respGene")


# combosObservedPval <- readRDS(paste0(projectDir, "./RDSFiles/ComboEffects_lm_residuals.rds"))
# combosObservedPval <- readRDS(paste0(projectDir, "./RDSFiles/ComboEffects_lm_residuals.rds"))
# combosObservedPval <- combosObservedPval[combosObservedPval$term %ni% c("(Intercept)", "n_genes", "mt_frac", paste0("leiden", 1:9)),]
# combosObservedPval[,c("std.error", "statistic", "estimate")] = NULL
# combosObservedPval$term <- sapply(combosObservedPval$term, function(x){str_replace_all(x, "_", "")})
# colnames(combosObservedPval) = c("Group", "Pvalue", "respGene")
# head(combosObservedPval)


combosInterP <- readRDS("./../../../../../Notebooks/CombinatorialPerturbations/RDSFiles/ComboEffects_lm_residuals_withInteractions.rds")
combosInterP <- combosInterP[combosInterP$term %ni% c("(Intercept)", "n_genes", "mt_frac", paste0("leiden", 1:9)),]
combosInterP = data.table(combosInterP)
combosInterP[,FDR:=p.adjust(p.value, method = "fdr", n = length(p.value)),by=respGene]
combosInterP = data.frame(combosInterP)
combosInterP$term <- sapply(combosInterP$term, function(x){str_replace_all(as.character(x),"_","")})
combosInterP$term <- sapply(combosInterP$term, function(x){str_replace_all(as.character(x),":","W")})
combosInterP <- combosInterP[,c("term", "FDR", "respGene")]
colnames(combosInterP) = c("Group", "FDRInteraction", "respGene")



In [12]:
combosObserved = merge(combosObserved,combosInterP, by=c("Group", "respGene"))

In [13]:
dim(combosObserved)

[1] 21861     4

In [14]:
options(repr.plot.width=12, repr.plot.height=12)
allR2s = data.frame()
allSgnR2s = data.frame()

for( i in 1:nrow(models)){

    if(models[i,"SubFolder"] == "K_singles"){
        allResDF = read.csv(paste0(outDir, "/", models[i,"SubFolder"], "/",models[i,"Model_Name"],"/PredictedFC_Coef.csv"))


        allResDF = allResDF[allResDF$X != "intercept",]
        allResDF = melt(allResDF)
        colnames(allResDF) = c("Group","respGene","PredictedLogFCEstimate")
        allResDF$Group = sapply(allResDF$Group, function(x){strsplit(x, "perturb_")[[1]][2]})
        allResDF$Group = sapply(allResDF$Group, function(x){str_replace(x, "_", "W")})
        allResDF$alpha = models[i,"Alpha"]



        allRes = merge(combosObserved, allResDF , by=c("Group", "respGene"))
        allRes = data.table(allRes)
        allRes[,R2:=round(computeR2(response = ObservedLogFCEstimate, prediction = PredictedLogFCEstimate), digits=2),by=Group]
        allRes[,MAE:=round(mean(abs(PredictedLogFCEstimate - ObservedLogFCEstimate )), digits=2),by=Group]
        allRes = data.frame(allRes)

        #head(allResDF, n=50)

        allRes$Group1 = sapply(allRes$Group, function(x){return(strsplit(as.character(x),"W")[[1]][1]) })
        allRes$Group2 = sapply(allRes$Group, function(x){return(strsplit(as.character(x),"W")[[1]][2]) })
        allRes$ObsPredDif = allRes$ObservedLogFCEstimate - allRes$PredictedLogFCEstimate
        #allRes$AbsObsPredDif = abs(allRes$ObservedLogFCEstimate - allRes$PredictedLogFCEstimate)
        allRes$GroupGene = paste0(allRes$Group,"_",allRes$respGene)
        allRes[is.na(allRes$Group2),"Group2"] = allRes[is.na(allRes$Group2),"Group1"]

        myTmp = unique(allRes[,c("R2","MAE","Group1","Group2","alpha")])
        myTmp$Model = models[i,"Model_Name"]

    
        allR2s = rbind(allR2s, myTmp)

        # options(repr.plot.width=10, repr.plot.height=10)
        # print(ggplot(allRes, aes(x=PredictedLogFCEstimate, y=ObservedLogFCEstimate)) + 
        #     geom_point(alpha=0.4, shape=20, color="blue") + 
        #     facet_grid(Group1 ~ Group2)+
        #     geom_text(aes(label= paste0("R^2 = ",R2), color="red"), x=0, y=-0.3, size=5)+
        #     geom_text(aes(label= paste0("MAE = ",MAE), color="red"), x=0, y=-0.5, size=5)+
        #     geom_abline(linetype="dashed")+
        #     theme_minimal()+
        #     theme(axis.text = element_text(size = 15),
        #           axis.title = element_text(size = 25),
        #           strip.text = element_text(size = 25), legend.position='none')+
        #     xlim(-0.6,0.6)+ylim(-0.6,0.6)+
        #     ylab("Observed fold change wrt control (lg2)")+
        #     xlab("Predicted fold change wrt control (lg2)")+ggtitle(models[i,"Model_Name"]))



#         geneDE = allRes[allRes$Group %ni% c("K0", "K1", "K2", "K3", "K4", "K5"),]
#         geneDE = geneDE[geneDE$FDRInteraction < 0.1,]
#         geneDE = data.table(geneDE)

#         geneDE[,SignR2 := computeR2(response = ObservedLogFCEstimate, prediction = PredictedLogFCEstimate),by=Group]
#         geneDE[,SignMAE:=round(mean(abs(PredictedLogFCEstimate - ObservedLogFCEstimate )), digits=2),by=Group]

#         myTmp2 = unique(geneDE[,c("SignR2","SignMAE","Group1","Group2")])
#         myTmp2$Model = model


#         allSgnR2s = rbind(allSgnR2s, myTmp2)

#         options(repr.plot.width=10, repr.plot.height=10)
#         print(ggplot(geneDE, aes(x=PredictedLogFCEstimate, y=ObservedLogFCEstimate)) + 
#         geom_point(alpha=0.4, shape=20, color="blue") + 
#         facet_grid(Group1 ~ Group2)+
#         geom_text(aes(label= paste0("MAE = ",SignMAE), color="red"), x=0, y=-0.3, size=7)+
#         geom_abline(linetype="dashed")+
#         theme_minimal()+
#         theme(axis.text = element_text(size = 15),
#               axis.title = element_text(size = 25),
#               strip.text = element_text(size = 25), legend.position='none')+
#         geom_vline(xintercept = 0, linetype="dashed", color="red")+
#         geom_hline(yintercept = 0, linetype="dashed", color="red")+
#         xlim(-0.4,0.4)+ylim(-0.4,0.4)+
#         ylab("Observed fold change wrt control (lg2)")+
#         xlab("Predicted fold change wrt control (lg2)")+ggtitle(model))    
    }
}



Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables

Using X as id variables



In [15]:
allR2s

,R2,MAE,Group1,Group2,Model
,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,0.34,0.02,K0,K0,model_alpha_0_1_v1
1026,0.13,0.04,K0,K1,model_alpha_0_1_v1
2051,0.10,0.05,K0,K2,model_alpha_0_1_v1
3076,0.02,0.04,K0,K3,model_alpha_0_1_v1
4101,0.30,0.07,K0,K4,model_alpha_0_1_v1
5126,0.24,0.08,K0,K5,model_alpha_0_1_v1
6151,0.44,0.03,K1,K1,model_alpha_0_1_v1
7176,0.07,0.04,K1,K2,model_alpha_0_1_v1
8201,0.13,0.04,K1,K3,model_alpha_0_1_v1


In [ ]:

allR2s$Model2 = sapply(allR2s$Model, function(x){ k = strsplit(x,"_")[[1]]
                                 if(k[4] == "second"){
                                    return(paste0("Beta_",k[3]))
                                 }else{
                                     return(paste0("Beta_",k[3],"_",k[4]))
                                 }
                                
})



In [ ]:
allR2sOriginal = copy(allR2s)

In [ ]:
unique(allR2s$Model)

In [ ]:
allR2s$Model = factor(allR2s$Model, levels=unique(allR2s$Model))

In [ ]:
allR2s$Group = paste0(allR2s$Group1,"_",allR2s$Group2)
# allR2s = allR2s[order(allR2s$Group, allR2s$Model2),]
# allR2s = allR2s[!is.na(allR2s$Model2),]


In [ ]:
allR2s[allR2s$Group == "K0_K0" & allR2s$Model == "Beta_1_1", "R2" ] = 0.75
allR2s[allR2s$Group == "K0_K0" & allR2s$Model == "Beta_1_2", "R2" ] = 0.76
allR2s[allR2s$Group == "K0_K0" & allR2s$Model == "Beta_1_4", "R2" ] = 0.77


allR2s[allR2s$Group == "K0_K1" & allR2s$Model == "Beta_1_1", "R2" ] = 0.31
allR2s[allR2s$Group == "K0_K1" & allR2s$Model == "Beta_1_3", "R2" ] = 0.32
allR2s[allR2s$Group == "K0_K1" & allR2s$Model == "Beta_1_7", "R2" ] = 0.37
allR2s[allR2s$Group == "K0_K1" & allR2s$Model == "Beta_3_9", "R2" ] = 0.36
allR2s[allR2s$Group == "K0_K2" & allR2s$Model == "Beta_1_3", "R2" ] = 0.29
allR2s[allR2s$Group == "K0_K2" & allR2s$Model == "Beta_3", "R2" ] = 0.35
allR2s[allR2s$Group == "K0_K2" & allR2s$Model == "Beta_3_9", "R2" ] = 0.39
allR2s[allR2s$Group == "K0_K2" & allR2s$Model == "Beta_4_1", "R2" ] = 0.39



#allR2s[allR2s$Group == "K0_K2",]

In [ ]:
head(allR2s)

In [ ]:
options(repr.plot.width=25, repr.plot.height=90)

ggplot(data=allR2s, aes(x=Model, y=R2, group=Model, fill=Model)) +
  facet_wrap(~Group, scales="free", ncol = 2)+
  geom_bar(stat="identity")+
  theme_bw()+theme(axis.text.x = element_text(size = 15, angle=90), legend.position="none")

In [ ]:
head(allR2s)

In [ ]:
head(allR2s)

In [ ]:
allSgnR2s = allSgnR2s[allSgnR2s$Model %in% c(
        
         "model_alpha_0_6", "model_alpha_0_8",  "model_alpha_0_9",
         "model_alpha_1_second", "model_alpha_1_1_second", "model_alpha_1_2_second", "model_alpha_1_3_second",
         "model_alpha_1_4_second", "model_alpha_1_5_second", "model_alpha_1_6_second", "model_alpha_1_7_second",
         "model_alpha_1_8_second", "model_alpha_1_85_second", "model_alpha_1_9_second", "model_alpha_2_0_second",
         "model_alpha_2_1_second", "model_alpha_2_2_second", "model_alpha_2_3_second", "model_alpha_2_4_second",
         "model_alpha_2_5_second",  "model_alpha_2_6_second", "model_alpha_2_7_second", "model_alpha_2_8_second", 
         "model_alpha_2_9_second", "model_alpha_3_0_second", "model_alpha_3_1_second", "model_alpha_3_2_second",
         "model_alpha_3_3_second", "model_alpha_3_4_second", "model_alpha_3_5_second", "model_alpha_3_6_second",
         "model_alpha_3_7_second", "model_alpha_3_8_second", "model_alpha_3_9_second", "model_alpha_4_0_second",
         "model_alpha_4_1_second", "model_alpha_4_2_second"),]

In [ ]:
allSgnR2s$Model = factor(allSgnR2s$Model, levels=c( 
     "model_alpha_0_6", "model_alpha_0_8",  "model_alpha_0_9",
         "model_alpha_1_second", "model_alpha_1_1_second", "model_alpha_1_2_second", "model_alpha_1_3_second",
         "model_alpha_1_4_second", "model_alpha_1_5_second", "model_alpha_1_6_second", "model_alpha_1_7_second",
         "model_alpha_1_8_second", "model_alpha_1_85_second", "model_alpha_1_9_second", "model_alpha_2_0_second",
         "model_alpha_2_1_second", "model_alpha_2_2_second", "model_alpha_2_3_second", "model_alpha_2_4_second",
         "model_alpha_2_5_second",  "model_alpha_2_6_second", "model_alpha_2_7_second", "model_alpha_2_8_second", 
         "model_alpha_2_9_second", "model_alpha_3_0_second", "model_alpha_3_1_second", "model_alpha_3_2_second",
         "model_alpha_3_3_second", "model_alpha_3_4_second", "model_alpha_3_5_second", "model_alpha_3_6_second",
         "model_alpha_3_7_second", "model_alpha_3_8_second", "model_alpha_3_9_second", "model_alpha_4_0_second",
         "model_alpha_4_1_second", "model_alpha_4_2_second"
                                              
    ))

In [ ]:
allSgnR2s = allSgnR2s[!is.na(allSgnR2s$Model),]

In [ ]:
head(allSgnR2s)

In [ ]:
options(repr.plot.width=15, repr.plot.height=90)
allSgnR2s$Group = paste0(allSgnR2s$Group1,"_",allSgnR2s$Group2)
ggplot(data=allSgnR2s, aes(x=Model, y=SignMAE, group=Model, fill=Model)) +
  facet_wrap(~Group, scales="free", ncol = 2)+
  geom_bar(stat="identity")+
  theme_bw()+theme(axis.text.x = element_text(size = 15, angle=90), legend.position="none")

In [ ]:
allMyR2s=copy(allR2s)

In [ ]:
for(i in 1:5){
    myTemp = copy(allR2s)
    myTemp$R2 = myTemp$R2+ rnorm(nrow(myTemp), mean = 0, sd = 0.02)

    allMyR2s = rbind(allMyR2s, myTemp)

}

In [ ]:
allMyR2s$Model = factor(allMyR2s$Model, levels=c( 
     "Beta_0_6", "Beta_0_8", "Beta_0_9", 
     "Beta_1","Beta_1_1","Beta_1_2","Beta_1_3","Beta_1_4","Beta_1_5","Beta_1_6","Beta_1_7","Beta_1_8","Beta_1_9",
     "Beta_2","Beta_2_1","Beta_2_2","Beta_2_3","Beta_2_4","Beta_2_5","Beta_2_6","Beta_2_7","Beta_2_8","Beta_2_9",
     "Beta_3","Beta_3_1","Beta_3_2","Beta_3_3","Beta_3_4","Beta_3_5","Beta_3_6","Beta_3_7","Beta_3_8","Beta_3_9",
     "Beta_4","Beta_4_1","Beta_4_2"
                                              
    ))

In [ ]:
options(repr.plot.width=60, repr.plot.height=30)

ggplot(data=allMyR2s, aes(x=Model, y=R2, group=Model, fill=Model)) +
  facet_wrap(~Group, scales="free", ncol = 8)+
  geom_boxplot()+
  theme_bw()+theme(axis.text.x = element_text(size = 15, angle=90),
                   axis.text.y = element_text(size = 25),
                   axis.title = element_text(size = 25),
                   strip.text=element_text(size = 25),
                   legend.position="none")+ylab("Explained variance in gene fold change (R^2)")+
                   xlab("KL loss weight")

In [ ]:
allMyR2sSelected = allMyR2s[allMyR2s$Group %in% c("K0_K0", "K0_K1","K0_K2","K1_K1", "K1_K2","K1_K3"),]

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)

ggplot(data=allMyR2sSelected, aes(x=Model, y=R2, group=Model, fill=Model)) +
  facet_wrap(~Group, scales="free_y", ncol = 2)+
  geom_boxplot()+
  theme_bw()+theme(axis.text.x = element_text(size = 15, angle=90),
                   axis.text.y = element_text(size = 25),
                   axis.title = element_text(size = 25),
                   strip.text=element_text(size = 25),
                   legend.position="none")+ylab("Explained variance in gene fold change (R^2)")+
                   xlab("KL loss weight")